In [1]:
import importlib
import sqlite3
import pandas as pd
from lib import kpi

In [15]:
importlib.reload(kpi)

<module 'lib.kpi' from '/Users/nxo/Workspace/GitHub/arcsaef/lib/kpi.py'>

In [53]:
data           = kpi.load_data()
rpt_config     = kpi.get_rpt_args()
saef_library   = kpi.get_saef_library()
responses_json = kpi.split_response(data[0])
ppl_collection = kpi.person_construct(responses_json, data[2], rpt_config[0])
proj_saef      = kpi.project_construct(responses_json, ppl_collection[0])
buckets        = kpi.matched_library(saef_library, ppl_collection[1])
templates      = kpi.load_templates()
meta_bucket    = buckets[0]
bucket         = buckets[1]
ppl_saef       = dict(sorted(ppl_collection[0].items(), key = lambda x: x[1].get('LastName')))
ppl_hash       = ppl_collection[1]
bulk_responses = data[0]
biblio         = data[1]
scopus         = data[2]
rpt_year       = rpt_config[0]
organisations  = rpt_config[1]
org_shortnames = list(organisations.keys())
proj_saef_nohold = proj_saef[proj_saef.Status != 'On hold']

In [54]:
# 1.remove empty rows
bucket = bucket[bucket['id_person'].isna() == False]
# 2. Make parsing more straight forward
for prsn in ppl_saef:
    if ppl_saef[prsn]['Gender'] == 'Non-binary/Gender diverse' or  ppl_saef[prsn]['Gender']  == 'Prefer not to say':
        ppl_saef[prsn]['Gender'] = 'Other'
# 3. Exclude: Nicole Webster, 027E2DEA-DB06-3946-B8C8-E053EF8E09F0
del ppl_saef['027E2DEA-DB06-3946-B8C8-E053EF8E09F0']; del ppl_hash['NicoleWebster']
# 4. Remove leading/trialing whitespace from  blibliographic entry.
biblio['Biblio'] = biblio['Biblio'].str.strip()
# 5. Jump into your pedantry
for prsn in ppl_saef:
    if ppl_saef[prsn]['Position'] != 'PhD Student' and  ppl_saef[prsn]['Position'] != 'Masters Student' and \
       ppl_saef[prsn]['Position'] != 'Honours Student':
        ppl_saef[prsn]['StudentProjectTitle'] = 'Not applicable'

In [74]:
con = sqlite3.connect(":memory:")
cur = con.cursor()
cur.execute("create table logistics (ID_Logistics, FieldSeason, PlanningStatus, Location, ReqPersonTotal, FinalPersonTotal, LeaveDate, ReturnDate, FinalPersonDays, FieldStatus)")
cur.execute("create table people_logistics (IDf_Logistics, IDf_Person, FullName, LeaveDate, ReturnDate, PersonDays, CareerStage, Gender, Org)")
cur.execute("create table logistics_projects (IDf_Logistics, IDf_Project, IDf_Person, ProjectCode, ProjectPercentage)")

In [75]:
# populate logistics table
for x in responses_json['Logistics_Detail']['data']:
    cur.execute("insert into logistics values (?,?,?,?,?,?,?,?,?,?)", 
          (x['fieldData']['ID_Logistics'],    x['fieldData']['FieldSeason'], \
           x['fieldData']['PlanningStatus'],  x['fieldData']['Location'], \
           x['fieldData']['ReqPersonTotal'],  x['fieldData']['FinalPersonTotal'], \
           x['fieldData']['LeaveDate'],       x['fieldData']['ReturnDate'], \
           x['fieldData']['FinalPersonDays'], x['fieldData']['FieldStatus']) )

In [76]:
# create a list of project(s) percentage per expeditioner  
pl = {}
for p in responses_json['logistics_Projects']['data']:
    # Active members, excl. Governance Committee memebers
    if p['fieldData']['IDf_Person'] != '' and  p['fieldData']['ProjectPercentage'] != '':
        if pl.get(p['fieldData']['IDf_Person']):
            pl[p['fieldData']['IDf_Person']].append(f"{p['fieldData']['Projects 3::ProjectCode']}, ({p['fieldData']['ProjectPercentage']}%)")
        else:
            pl[p['fieldData']['IDf_Person']] = [f"{p['fieldData']['Projects 3::ProjectCode']}, ({p['fieldData']['ProjectPercentage']}%)"] 

# populate people_Logistics table
for x in responses_json['people_Logistics']['data']:
    cur.execute("insert into people_logistics values (?,?,?,?,?,?,?,?,?)", 
          (x['fieldData']['IDf_logistics'],       x['fieldData']['IDf_Person'], \
           x['fieldData']['People::FullName'],    x['fieldData']['DateFrom'], \
           x['fieldData']['DateTo'],              x['fieldData']['PersonDays'], \
           x['fieldData']['People::CareerStage'], x['fieldData']['People::Gender'], \
           x['fieldData']['People::OrganisationAbbreviation']) )

In [77]:
alter_statement  = "ALTER TABLE  people_Logistics  ADD COLUMN  Projects  TEXT";
cur.execute(alter_statement)
con.commit()

update_statement = "UPDATE people_Logistics SET Projects=? WHERE IDf_Person=?"
for p in pl:
    cur.execute(update_statement, (', '.join(pl.get(p)), p))
    con.commit()

In [78]:
# populate logistics_projects table
for x in responses_json['logistics_Projects']['data']:
    cur.execute("insert into logistics_projects values (?,?,?,?,?)", 
          (x['fieldData']['IDf_Project'],       x['fieldData']['IDf_Logistics'], \
           x['fieldData']['IDf_Person'],        x['fieldData']['Projects 3::ProjectCode'], \
           x['fieldData']['ProjectPercentage']) )

In [86]:
# Gender breakdown of completed expeditions
pd.read_sql_query("select a.Gender, count(a.Gender) as Total from people_logistics a \
                  join logistics b on a.IDf_Logistics = b.ID_Logistics \
                  where b.FieldStatus ='Completed' and a.CareerStage not like 'Field%'\
                  group by a.Gender \
                  union \
                  select '-', count(a.Gender) as Total from people_logistics a \
                  join logistics b on a.IDf_Logistics = b.ID_Logistics \
                  where b.FieldStatus ='Completed' and a.CareerStage not like 'Field%' \
                  order by 2" , con )


,Gender,Total
0,Non-binary,1
1,,4
2,Man,12
3,Woman,15
4,-,32


In [80]:
# Career stage breakdown of completed projects
pd.read_sql_query("select a.CareerStage, count(a.CareerStage) as Total from people_logistics a \
            join logistics b on a.IDf_Logistics = b.ID_Logistics where b.FieldStatus ='Completed' \
            and  b.FieldSeason = '2021/22' and a.CareerStage not like 'Field%'\
            group by a.CareerStage \
            union \
            select '-', count(a.CareerStage) as Total from people_logistics a \
            join logistics b on a.IDf_Logistics = b.ID_Logistics where b.FieldStatus ='Completed' \
            and  FieldSeason = '2021/22' and a.CareerStage not like 'Field%' \
            order by 2", con )


,CareerStage,Total
0,Early Career (<5yrs post PhD),1
1,Senior Researcher,1
2,Student,1
3,Mid Career (5-15yrs post PhD),2
4,Professional,2
5,-,7


In [85]:
# Career stage breakdown of completed projects
                #   --100.0 * count(a.Org) / (select count(*) from people_logistics) \
pd.read_sql_query("select a.Org as Organisation, count(a.Org) as Total, \
                  round(100.0 * count(a.Org) / (select count(a.Org) from people_logistics a \
                  join logistics b on a.IDf_Logistics = b.ID_Logistics \
                  where b.FieldStatus ='Completed' and a.CareerStage not like 'Field%' ), 0) as Percent \
                  from people_logistics a \
                  join logistics b on a.IDf_Logistics = b.ID_Logistics \
                  where b.FieldStatus ='Completed' and a.CareerStage not like 'Field%' \
                  group by a.Org \
                  union \
                  select '-', count(a.Org), '100.0' from people_logistics a \
                  join logistics b on a.IDf_Logistics = b.ID_Logistics \
                  where b.FieldStatus ='Completed' and a.CareerStage not like 'Field%' \
                  order by 3 desc", con )


,Organisation,Total,Percent
0,-,32,100.0
1,,10,31.0
2,Monash,7,22.0
3,UOW,6,19.0
4,QUT,5,16.0
5,AUT,3,9.0
6,La Trobe,1,3.0


In [90]:
con.close()

In [89]:
cur.execute("select * from  people_Logistics")
cur.fetchall()

[('6E843797-DF41-41CC-BEA0-95629B96B117',
  '18B831FC-B410-D040-ABE6-941BB56C51C9',
  'Caitlin Selfe',
  '2022-10-21',
  '2023-04-18',
  159,
  'Student',
  '',
  'QUT',
  'T2_P019, (100%)'),
 ('982AAC88-C059-47D4-B366-81EF8B9A0315',
  '10C00B70-4EC2-3B48-9073-957BB328D7B1',
  'Felicity McCormack',
  '2022-02-02',
  '2022-02-25',
  23,
  'Mid Career (5-15yrs post PhD)',
  'Woman',
  'Monash',
  'T1_P007, (100%)'),
 ('E561AE5A-BE76-41BF-AC96-CCF4718726B5',
  'E485E5D8-D06C-C646-98C8-758AD0BBCDA9',
  'Georgia Watson',
  '2022-02-02',
  '2022-02-25',
  23,
  'Professional',
  'Woman',
  'UOW',
  'T2_P022, (100%)'),
 ('B49D6C20-2ED2-404A-805D-39B7F6998762',
  '4C10171F-63C4-CB48-9C3E-A4E92BEE5439',
  'Jeremy  Bird',
  '2024-05-31',
  '2024-07-15',
  38,
  'Field Leader',
  'Man',
  'Monash',
  'T2_P019, (33%), T1_P004, (11%)'),
 ('3D75909A-B14A-4149-9551-4C5EB24C0722',
  '76AB4EA1-E845-DD44-A427-D669BC8FAA6E',
  'Johan Barthélemy',
  '2022-12-20',
  '2023-02-04',
  36,
  'Mid Career (5-15y